In [1]:
from emgdecomp.decomposition import EmgDecomposition
from emgdecomp.parameters import EmgDecompositionParams
from tools import flatten_signal, discardChannels, bandpass_filter
from scipy.io import loadmat
import numpy as np

In [2]:
path = r"F:\7_MU Edit\opensource data\GM_10_seg.mat"
SIG = loadmat(path)["SIG"]
ch2discard = loadmat(path)["discardChannelsVec"]

flattened_emg = flatten_signal(SIG)
channels_to_discard = flatten_signal(ch2discard)
channels_to_discard = channels_to_discard[1:]

flattened_emg = discardChannels(flattened_emg, channels_to_discard)
flattened_emg = bandpass_filter(flattened_emg)


# fs = loadmat(path)["fsamp"].item()
# fs = fs.astype(np.int64)
# print(type(fs))

params = EmgDecompositionParams(
    sampling_rate=float(loadmat(path)["fsamp"].item())
)

decomp = EmgDecomposition(params=params)
IPTs, MUid, MUpulses, Threshold= decomp.decompose(flattened_emg)

2025-05-22 06:18:26,646 - INFO - Computing covariance matrix...
2025-05-22 06:18:26,876 - INFO - Covariance computed. Calculating whitening matrix...
2025-05-22 06:18:27,241 - INFO - Whitened data created. Array: <emgdecomp._data.CpuDataManager object at 0x0000028D382D0D50>
2025-05-22 06:18:27,343 - INFO - ===== SOURCE INDEX 0
2025-05-22 06:18:27,343 - INFO - Initialization done using data index 54314, value 872.2307171302583
2025-05-22 06:18:27,477 - INFO - ICA iteration 1: 0.005709131897133624
2025-05-22 06:18:27,593 - INFO - ICA iteration 2: 0.0015866951165652843
2025-05-22 06:18:27,727 - INFO - ICA iteration 3: 0.001194603285719853
2025-05-22 06:18:27,843 - INFO - ICA iteration 4: 0.001478859231622498
2025-05-22 06:18:27,977 - INFO - ICA iteration 5: 0.002508749152493661
2025-05-22 06:18:28,093 - INFO - ICA iteration 6: 0.0055748761784018
2025-05-22 06:18:28,227 - INFO - ICA iteration 7: 0.015043614651659265
2025-05-22 06:18:28,360 - INFO - ICA iteration 8: 0.036100574865084556
202

In [3]:
print(MUid)
print(IPTs.shape)
print(MUpulses.shape)
print(Threshold)

['MU_1', 'MU_2', 'MU_3', 'MU_4', 'MU_5', 'MU_6', 'MU_7', 'MU_8', 'MU_9', 'MU_10', 'MU_11', 'MU_12', 'MU_13', 'MU_14', 'MU_15', 'MU_16', 'MU_17', 'MU_18']
(18, 64284)
(18,)
[42.80005913 49.34854818 45.8570216  45.85316539 32.17711848 23.79089085
  7.91843095 23.52357002 27.37625807 16.27094071 29.14638388 29.65081952
 12.02871561 10.26303292  6.84194655 26.01927487  6.77714583 27.94286613]


In [9]:
timings = decomp.MUST
print(timings.shape)
print(type(timings))
print(timings)
IPT = decomp.output_sources
print(IPT.shape)

(4443,)
<class 'numpy.ndarray'>
[(  3,  9984,  4.875     , 4.01826091e+01)
 (  3, 11662,  5.69433594, 2.49822871e+01)
 (  3, 12082,  5.89941406, 3.60709710e+01) ...
 (146, 59025, 28.82080078, 3.47662966e-04)
 (146, 60146, 29.36816406, 2.81321124e-04)
 (146, 60702, 29.63964844, 6.30338569e-04)]
(992, 16)


In [7]:
print(IPTs.shape)
print(IPTs)

(21, 64284)
[[ -88.17706654  -28.61541469  -77.6605501  ...   45.69424334
    -2.57500012  -61.16361427]
 [ -53.06786682  -80.5548785   -55.44947196 ...  -92.75912192
   -14.12348602    7.79984165]
 [  38.83059776   -4.22986725  -11.3682514  ...  -52.28719678
   -19.9762107    71.79067219]
 ...
 [  64.63125774   56.43618739   17.54380649 ...   35.16597352
   -83.02439285 -155.31111072]
 [ -65.28086563  -35.73577226  -34.23789376 ... -139.20368288
  -105.47369491   46.4647208 ]
 [ 198.52701883  101.30721454   91.96782052 ... -100.36234223
  -248.10105669 -283.62428806]]


In [22]:
import pandas as pd
MUPulses = []
print(pd.DataFrame(firings))
firings_df = pd.DataFrame(firings)
unique_sources = firings_df['source_idx'].unique()
for source in unique_sources:
    discharge_samples = firings_df[firings_df['source_idx'] == source]['discharge_samples'].astype(np.float64).tolist()
    MUPulses.append(discharge_samples)
print(MUPulses)
MUPulses = np.array(MUPulses, dtype="object")
fout_name = "out.mat"
import scipy.io as sio
result = {}
result["MUPulses"] = MUPulses
sio.savemat(fout_name, result)


      source_idx  discharge_samples  discharge_seconds  squared_amplitude
0              6               5781           2.822754          62.205345
1              6               6439           3.144043          81.400640
2              6               6631           3.237793          80.016210
3              6               6851           3.345215          99.059607
4              6               7041           3.437988          65.414444
...          ...                ...                ...                ...
5076           9              59507          29.056152          11.091645
5077           9              59801          29.199707          11.625841
5078           9              60208          29.398438          12.085599
5079           9              60575          29.577637          11.073098
5080           9              61450          30.004883          19.363901

[5081 rows x 4 columns]
[[5781.0, 6439.0, 6631.0, 6851.0, 7041.0, 7261.0, 7483.0, 7685.0, 8097.0, 8305.0, 8502.

In [13]:
last_ids = None
l = len(firings)
full_container = []
container = []
for i in range(l):
    # print(i)
    sub_firing_idx = firings[i][0]
    if sub_firing_idx != last_ids and last_ids is not None:
        # print(f"Start a new idx")
        print(f"Start sorting {sub_firing_idx}")
        print(f"Finish")
        full_container.append(container)
        container = []
        last_ids = sub_firing_idx
    container.append(firings[i][1])
    last_ids = sub_firing_idx

# full_container.pop(0)
# for i, s in enumerate(full_container):
#     print(f"The {i+1}th source")
#     print(f"Include {len(s)} spikes")
# # full_container = np.array(full_container)
# # print(full_container.shape)
# print(full_container)


Start sorting 2
Finish
Start sorting 7
Finish
Start sorting 10
Finish
Start sorting 1
Finish
Start sorting 12
Finish
Start sorting 3
Finish
Start sorting 13
Finish
Start sorting 14
Finish
Start sorting 15
Finish
Start sorting 16
Finish
Start sorting 17
Finish
Start sorting 18
Finish
Start sorting 19
Finish
Start sorting 20
Finish
Start sorting 11
Finish
Start sorting 8
Finish
Start sorting 0
Finish
Start sorting 4
Finish
Start sorting 5
Finish
Start sorting 9
Finish


In [32]:
print(decomp.output_sources.shape)

(992, 16)
